# Assignment 4 - Fixed & Stable Version
Includes corrected Ridge Regression GD.

## Q1 – Stable Ridge Regression with Gradient Descent

In [ ]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

# Generate synthetic correlated dataset
np.random.seed(42)
n = 500
x1 = np.random.randn(n)
x2 = x1 + np.random.normal(0, 0.1, n)
x3 = x2 + np.random.normal(0, 0.1, n)
x4 = x3 + np.random.normal(0, 0.1, n)
x5 = x4 + np.random.normal(0, 0.1, n)
x6 = x5 + np.random.normal(0, 0.1, n)
x7 = x6 + np.random.normal(0, 0.1, n)

X = np.vstack([x1,x2,x3,x4,x5,x6,x7]).T
y = 3*x1 + 2*x2 - x3 + 0.5*x4 + np.random.randn(n)

# Scale features + target
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

y = (y - y.mean()) / y.std()

def ridge_gd(X, y, lr, lam, epochs=2000, clip=1000):
    m, n = X.shape
    w = np.zeros(n)

    for _ in range(epochs):
        y_pred = X.dot(w)
        grad = -(2/m) * X.T.dot(y - y_pred) + 2 * lam * w
        grad = np.clip(grad, -clip, clip)
        w -= lr * grad

        if np.any(np.isnan(w)) or np.any(np.isinf(w)):
            return w, np.inf, -np.inf

    cost = np.mean((y - X.dot(w))**2) + lam * np.sum(w**2)
    return w, cost, r2_score(y, X.dot(w))

learning_rates = [0.0001, 0.001, 0.01]
lambdas = [1e-5, 1e-3, 0, 1, 10, 20]

results = []
for lr in learning_rates:
    for lam in lambdas:
        w, cost, r2 = ridge_gd(X_scaled, y, lr, lam)
        results.append((lr, lam, cost, r2))

df_results = pd.DataFrame(results, columns=["Learning Rate","Lambda","Cost","R2 Score"])
print(df_results)


    Learning Rate    Lambda      Cost  R2 Score
0          0.0001   0.00001  0.072259  0.927742
1          0.0001   0.00100  0.072389  0.927729
2          0.0001   0.00000  0.072258  0.927742
3          0.0001   1.00000  0.187694  0.906606
4          0.0001  10.00000  0.619125  0.605447
5          0.0001  20.00000  0.760600  0.417240
6          0.0010   0.00001  0.064277  0.935724
7          0.0010   0.00100  0.064428  0.935716
8          0.0010   0.00000  0.064276  0.935724
9          0.0010   1.00000  0.185963  0.917332
10         0.0010  10.00000  0.619125  0.605961
11         0.0010  20.00000  0.760600  0.417247
12         0.0100   0.00001  0.052029  0.947976
13         0.0100   0.00100  0.052532  0.947877
14         0.0100   0.00000  0.052023  0.947977
15         0.0100   1.00000  0.185963  0.917353
16         0.0100  10.00000  0.619125  0.605961
17         0.0100  20.00000  0.760600  0.417247


## Q2 – Hitters Dataset (User must add dataset manually)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error

df = pd.read_csv("Hitters.csv")

df = df.dropna()

df = pd.get_dummies(df, drop_first=True)

X = df.drop("Salary", axis=1)
y = df["Salary"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
ridge = Ridge(alpha=0.5748)
lasso = Lasso(alpha=0.5748)

lr.fit(X_train_scaled, y_train)
ridge.fit(X_train_scaled, y_train)
lasso.fit(X_train_scaled, y_train)

models = {
    "Linear Regression": lr,
    "Ridge Regression": ridge,
    "Lasso Regression": lasso
}

for name, model in models.items():
    y_pred = model.predict(X_test_scaled)
    print("----", name, "----")
    print("R2 Score:", r2_score(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred))
    print()


---- Linear Regression ----
R2 Score: 0.16769360190025306
RMSE: 150540.93304991833

---- Ridge Regression ----
R2 Score: 0.16975236618374923
RMSE: 150168.56021117172

---- Lasso Regression ----
R2 Score: 0.14682108115827563
RMSE: 154316.18787767767



## Q3 – RidgeCV & LassoCV

In [ ]:

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import RidgeCV, LassoCV

data = fetch_california_housing()
X = data.data
y = data.target

ridge_cv = RidgeCV(alphas=[0.1,1,10]).fit(X,y)
lasso_cv = LassoCV(alphas=[0.1,1,10]).fit(X,y)

print(ridge_cv.alpha_, lasso_cv.alpha_)


10.0 0.1


## Q4 – Multiclass Logistic Regression (One-vs-Rest)

In [ ]:

from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
X = iris.data
y = iris.target

def sigmoid(z):
    return 1/(1+np.exp(-z))

def train_binary(X, y, lr=0.01, epochs=3000):
    m, n = X.shape
    w = np.zeros(n)
    for _ in range(epochs):
        p = sigmoid(X.dot(w))
        grad = (1/m) * X.T.dot(p - y)
        w -= lr * grad
    return w

weights = []
for cls in np.unique(y):
    y_bin = (y == cls).astype(int)
    w = train_binary(X, y_bin)
    weights.append(w)

print(weights)


[array([ 0.42463804,  1.35703565, -2.09708363, -0.94996822]), array([ 0.26346122, -1.00975127,  0.38051526, -0.59756239]), array([-1.2947823 , -1.19007315,  1.85162845,  1.52188485])]
